In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('Cluster/df_faenas_y_desaladoras_y_puertos_y_estaciones.csv')

In [10]:
df = df.drop(columns = ['categorias_10km'])

In [ ]:
from IPython.display import HTML, display

# Scrollable view of the entire DataFrame
display(HTML(
    '<div style="max-height:600px; overflow:auto; border:1px solid #ccc;">'
    + df.to_html(index=False)
    + '</div>'
))

,RutEmpresa,NombreEmpresa,CategoriaFaena,RegionInstalacion,TipoInstalacion,Cota,Huso,Estado,Latitud,Longitud,...,Tipo de Planta,Uso/Aplicación,Capacidad (Valor),Año de Puesta en Marcha,travel_dist_km,travel_time_desal,portName,harborSize,travel_time_min,stationName
0,61703000-4,EMPRESA NACIONAL DE MINERIA (ENAMI),CATEGORIA B,ATACAMA,TALLERES Y MAESTRANZA,538,19,ACTIVA,-28.566725,-70.740064,...,Plantas deslinizadoras de agua de mar con capa...,Industrial,70.0,1997,2.751130,58.666667,Huasco,V,6.401667,Est VALLENAR
1,76125921-0,CUPRUM RESOURCES CHILE SPA,CATEGORIA D,COQUIMBO,EXPLORACION DE SUPERFICIE,1200,19,INACTIVA,-29.432920,-70.683699,...,Iniciativas y proyectos en evaluación preliminar,Minería,970.0,ND,25.212951,137.216667,Coquimbo,V,44.311667,Est TRES CRUCES
2,76362163-4,MINERA RUPERTO PATRICIO ORTIZ PALOMINOS,CATEGORIA D,LIBERTADOR GENERAL BERNARDO OHIGGINS,MINA SUBTERRANEA,134,19,ABANDONADA,-34.677189,-71.498091,...,Plantas deslinizadoras de agua de mar con capa...,Industrial,30.0,2008,15.116433,247.316667,Puerto San Antonio,S,19.346667,Est SANTA CRUZ
3,6891906-1,GARY NELSON MERCADO MALBRAN,CATEGORIA D,ATACAMA,MINA SUBTERRANEA,531,19,ACTIVA,-26.411292,-70.414104,...,Plantas deslinizadoras de agua de mar con capa...,Minería,120.0,2014,9.329149,41.766667,Puerto Chanaral,V,9.081667,Est SALADO
4,76448510-6,SLM CASUALIDAD PRIMERA DE EL MANZANO,CATEGORIA D,COQUIMBO,MINA SUBTERRANEA,917,19,IRREGULAR OPERATIVA,-30.204069,-71.082232,...,Plantas en construcción o prontas a entrar en ...,Minería,400.0,2023,15.026041,182.316667,Coquimbo,V,26.790000,Est ANDACOLLO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21153,78131832-9,BALFENI SOLUTIONS SPA,CATEGORIA D,COQUIMBO,CAMPAMENTO,1277,19,EN REVISIÓN DE PROYECTO,-31.216132,-71.267759,...,Plantas en construcción o prontas a entrar en ...,Minería,400.0,2023,21.812106,88.783333,Coquimbo,V,57.861667,Est PAMA
21154,78131832-9,BALFENI SOLUTIONS SPA,CATEGORIA D,COQUIMBO,MINA SUBTERRANEA,1199,19,EN REVISIÓN DE PROYECTO,-31.220795,-71.265226,...,Plantas en construcción o prontas a entrar en ...,Minería,400.0,2023,21.812106,88.783333,Coquimbo,V,57.861667,Est PAMA
21155,6188182-4,EDILIO GOMEZ OYARZUN,CATEGORIA D,COQUIMBO,MINA SUBTERRANEA,458,19,EN REVISIÓN DE PROYECTO,-31.548972,-71.343902,...,Plantas en construcción o prontas a entrar en ...,Minería,400.0,2023,17.028262,71.300000,Bahia Quintero (Ventanas),V,59.890000,Est CUZ CUZ
21156,78110627-5,C & C MINERIA SpA,CATEGORIA D,ANTOFAGASTA,POLVORIN,3250,19,EN REVISIÓN DE PROYECTO,-23.044930,-68.557016,...,Plantas menores en operación,Industrial,5.0,2005,75.570945,263.763333,Antofagasta,M,109.315000,Est CALAMA
